In [ ]:
from scrapy import signals

# useful for handling different item types with a single interface
from itemadapter import is_item, ItemAdapter


class NarkotikaSpiderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the spider middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_spider_input(self, response, spider):
        # Called for each response that goes through the spider
        # middleware and into the spider.

        # Should return None or raise an exception.
        return None

    def process_spider_output(self, response, result, spider):
        # Called with the results returned from the Spider, after
        # it has processed the response.

        # Must return an iterable of Request, or item objects.
        for i in result:
            yield i

    def process_spider_exception(self, response, exception, spider):
        # Called when a spider or process_spider_input() method
        # (from other spider middleware) raises an exception.

        # Should return either None or an iterable of Request or item objects.
        pass

    def process_start_requests(self, start_requests, spider):
        # Called with the start requests of the spider, and works
        # similarly to the process_spider_output() method, except
        # that it doesn’t have a response associated.

        # Must return only requests (not items).
        for r in start_requests:
            yield r

    def spider_opened(self, spider):
        spider.logger.info("Spider opened: %s" % spider.name)


class NarkotikaDownloaderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the downloader middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_request(self, request, spider):
        # Called for each request that goes through the downloader
        # middleware.

        # Must either:
        # - return None: continue processing this request
        # - or return a Response object
        # - or return a Request object
        # - or raise IgnoreRequest: process_exception() methods of
        #   installed downloader middleware will be called
        return None

    def process_response(self, request, response, spider):
        # Called with the response returned from the downloader.

        # Must either;
        # - return a Response object
        # - return a Request object
        # - or raise IgnoreRequest
        return response

    def process_exception(self, request, exception, spider):
        # Called when a download handler or a process_request()
        # (from other downloader middleware) raises an exception.

        # Must either:
        # - return None: continue processing this exception
        # - return a Response object: stops process_exception() chain
        # - return a Request object: stops process_exception() chain
        pass

    def spider_opened(self, spider):
        spider.logger.info("Spider opened: %s" % spider.name)

In [ ]:
from itemadapter import ItemAdapter


class NarkotikaPipeline:
    def process_item(self, item, spider):
        return item

In [ ]:
import scrapy

class PidanaUmumSpider(scrapy.Spider):
    name = "scraper"
    allowed_domains = ["putusan3.mahkamahagung.go.id"]
    start_urls = ["https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-jombang/kategori/narkotika-dan-psikotropika-1.html"]
    
    def parse(self, response):
        next_page = response.xpath('//*[@id="tabs-1"]/div[3]/nav/ul/li[5]/a[2]/@href').extract_first()
        if next_page:
            yield scrapy.Request(next_page, callback=self.parse_first)
            
    def parse_first(self, response):
        per_link = response.css('div.spost.clearfix strong a')
        for page in per_link:
            link = page.attrib['href']
            if link:
                print(link)
                yield scrapy.Request(link, callback=self.parse_second)
    
    def parse_second(self, response):
        for selector in response.css('tbody'):
            yield {
                'Tanggal': selector.css('tr td h2::text')[1].get(),
                'Penuntut Umum': selector.css('tr td h2::text')[2].get(),
                'Terdakwa': selector.css('tr td h2::text')[4:].get(),
                'Nomor Putusan': selector.css('tr td')[2].get(),
                'Kata Kunci': selector.css('tr td')[8].get(),
                'Tanggal Register': selector.css('tr td')[12].get(),
                'Lembaga Peradilan': 'PN JOMBANG',
                'Hakim Ketua': selector.css('tr td')[18].get(),
                'Hakim Anggota': selector.css('tr td')[20].get(),
                'Panitera': selector.css('tr td')[22].get(),
                'Catatan Amar': selector.css('tr td')[28].get(),                
            }